# **Photometric Redshift Estimation** 
**Utilizes a neural network to predict photometric redshift given a data set that contains photometric band magnitude values and known spectroscopic redshift.**

In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib as plt
from torch.utils.data.dataset import random_split
from torch.utils.data import TensorDataset, DataLoader
from pathlib import Path
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error


# Load data
#path = Path('/Users/katesautel/Documents/Seminar_2024/hsc_v6.csv')
path = Path('/Users/katesautel/Documents/Seminar_2024/rel_z.csv')
df = pd.read_csv(path)


# Split data into features (F) and target (T)
F = df.iloc[:-1, 0:5]
T = df.iloc[:-1, 5]


# Scale features to be between 0 and 1
scaler = MinMaxScaler()
F_scaled = scaler.fit_transform(F)


# Convert data from arrays into tensors
F_tensor = torch.FloatTensor(F_scaled)
T_tensor = torch.FloatTensor(T.values)


# Combine F and T into single dataset
data = TensorDataset(F_tensor, T_tensor)


# Determine training vs. test set split (50%/50%)
train_size = len(data) // 2
test_size = len(data) - train_size


# Randomly split data into train and test
train_data, test_data = random_split(dataset = data, lengths = [train_size, test_size])


# Initialize hyperparameters
num_input_features = 5
num_hidden_neurons = 128
num_hidden_layers = 5
num_epochs = 20
learning_rate = 0.001
size_batch = 32
momentum = 0.9


# Create DataLoader objects (batch and shuffle data)
train_dl = DataLoader(dataset = train_data,
                      batch_size = size_batch,
                      shuffle = True)
test_dl = DataLoader(dataset = test_data,
                     batch_size = size_batch,
                     shuffle = False)


# Define neural network
class NeuralNetwork(nn.Module):
    def __init__(self, num_input_features, 
                 num_hidden_neurons, 
                 num_hidden_layers):
        super(NeuralNetwork, self).__init__()
        self.input_layer = nn.Linear(num_input_features, 
                                     num_hidden_neurons)
        self.hidden_layers = nn.ModuleList(
            [nn.Linear(num_hidden_neurons, 
                       num_hidden_neurons) for n in range(num_hidden_layers)])
        self.output_layer = nn.Linear(num_hidden_neurons, 1)

    def forward(self, x):
        x = torch.relu(self.input_layer(x))
        for layer in self.hidden_layers:
            x = torch.relu(layer(x))
        x = self.output_layer(x)
        return x


# Initialize model
model = NeuralNetwork(num_input_features, num_hidden_neurons, num_hidden_layers)


# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum)

min_loss = 1
best_model = None
loss_data = []


# Train the model
for epoch in range(num_epochs):
    for inputs, targets in train_dl:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets.unsqueeze(1))
        loss.backward()
        optimizer.step()

    if loss.item() < min_loss:
        min_loss = loss.item()
        best_model = model.state_dict()

    if (epoch + 1) % 1 == 0:  # Print every epoch
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')
        loss_data.append(loss.item())
        print()


# Save the best model
if best_model is not None:
    torch.save(best_model, '/Users/katesautel/Documents/Seminar_2024/best_model.pth')


# Load best model
best_model = NeuralNetwork(num_input_features, num_hidden_neurons, num_hidden_layers)
best_model.load_state_dict(torch.load('/Users/katesautel/Documents/Seminar_2024/best_model.pth'))


# Evaluate model performance
best_model.eval()
with torch.no_grad():
    y_true = [] 
    y_pred = []

    for inputs, targets in test_dl:
        outputs = best_model(inputs)

        mse = mean_squared_error(targets.numpy(), outputs.numpy().squeeze())

        y_true.extend(targets.numpy())
        y_pred.extend(outputs.numpy().squeeze())


# Save results
y_true = np.array(y_true)
y_pred = np.array(y_pred)
results = pd.DataFrame({'Spectroscopic': y_true, 'Photometric': y_pred})


# Categorize outliers
results['Outlier'] = np.select([abs(results['Photometric'] - 
                                    results['Spectroscopic']) > 1,
                                abs(results['Photometric'] - 
                                    results['Spectroscopic']) > 0.15 * (1 + results['Spectroscopic'])],
                               ['CO', 'O'], 'NO')


# Export data
results.to_csv('/Users/katesautel/Documents/Seminar_2024/results.csv', index=False)

loss_data = pd.DataFrame({'Loss': loss_data})
loss_data.to_csv('/Users/katesautel/Documents/Seminar_2024/loss.csv', index=False)

print("MSE:", mse)

Epoch [1/20], Loss: 0.3881

Epoch [2/20], Loss: 0.3362

Epoch [3/20], Loss: 0.4337

Epoch [4/20], Loss: 0.2853

Epoch [5/20], Loss: 0.2614

Epoch [6/20], Loss: 0.2624

Epoch [7/20], Loss: 0.1691

Epoch [8/20], Loss: 0.2791

Epoch [9/20], Loss: 0.2067

Epoch [10/20], Loss: 0.1879

Epoch [11/20], Loss: 0.2397

Epoch [12/20], Loss: 0.0659

Epoch [13/20], Loss: 0.1377

Epoch [14/20], Loss: 0.1828

Epoch [15/20], Loss: 0.3347

Epoch [16/20], Loss: 0.2310

Epoch [17/20], Loss: 0.1797

Epoch [18/20], Loss: 0.2444

Epoch [19/20], Loss: 0.0770

Epoch [20/20], Loss: 0.1531

MSE: 0.048282005
